# Install required Packages

In [ ]:
!pip install transformers --quiet
!pip install gradio --quiet
!pip install bitsandbytes --quiet
!pip install torch torchvision torchaudio --quiet
!pip install huggingface_hub --quiet
!pip install huggingface_hub[hf_xet] --quiet
!pip install "datasets>=3.4.1" --quiet
!pip install pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.

# Importing required libraries

In [ ]:
import torch
import gradio as gr
import os
from datasets import load_dataset
import warnings
warnings.filterwarnings('ignore')

# Importing HF Token

In [ ]:
from google.colab import userdata
huggingface_user=userdata.get('HF_TOKEN')

In [ ]:
from huggingface_hub import whoami
print(whoami(token=huggingface_user))

{'type': 'user', 'id': '66957888f4d5f5d06ca36462', 'name': 'Suman2004', 'fullname': 'Suman Roy', 'email': 'sumanroy202400@gmail.com', 'emailVerified': True, 'canPay': False, 'periodEnd': 1748735999, 'isPro': False, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/66957888f4d5f5d06ca36462/BQzGRCRMfy4WL6NSsAICJ.jpeg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'sent_token', 'role': 'write', 'createdAt': '2025-05-14T17:59:22.556Z'}}}


In [ ]:
from huggingface_hub import login
login(token=huggingface_user)

# Loading the finetuned model

In [ ]:
from transformers import AutoProcessor, AutoModelForImageTextToText, TextStreamer
import torch

# Load your fine-tuned model and processor
model_id = "Suman2004/covid_chestXray_radiologist"
processor = AutoProcessor.from_pretrained(model_id,token=huggingface_user)
model = AutoModelForImageTextToText.from_pretrained(model_id,token=huggingface_user, torch_dtype=torch.float16).to("cuda")

# Launching the gradio app

### Prediction on Single

In [ ]:
from PIL import Image


# Load image (same format as during training)
image = Image.open("/content/Normal.jpg")

# Instruction text (same as during fine-tuning)
instruction = '''You are an expert radiologist. Analyze the provided chest X-ray image and classify the condition into:

- COVID-19
- Normal
- Pneumonia (non-COVID)

Respond with:
1. Classification: [Condition Name]
2. Justification: [Clinical Findings]'''

# Format as chat-style messages (same structure as training)
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": instruction},
            {"type": "image", "image": image}
        ]
    }
]

# Apply chat template to get proper prompt format
prompt = processor.tokenizer.apply_chat_template(messages, add_generation_prompt=True,tokenize=False)

# Preprocess with text + image
inputs = processor(
    text=prompt,
    images=image,
    return_tensors="pt"
).to("cuda", torch.float16)

# Stream output (optional but useful for long generation)
streamer = TextStreamer(processor.tokenizer, skip_prompt=True)

# Generate the model output
_ = model.generate(
    **inputs,
    streamer=streamer,
    max_new_tokens=300,
    use_cache=True,
    temperature=1.2,
    top_p=0.9
)


1. Classification: Normal
2. Justification: No focal consolidations or opacities observed. Lung fields appear clear.<|eot_id|>


### GRADIO APP

In [ ]:
from PIL import Image

# Define the instruction (same as during fine-tuning)
instruction = '''You are an expert radiologist. Analyze the provided chest X-ray image and classify the condition into:

- COVID-19
- Normal
- Pneumonia (non-COVID)

Respond with:
1. Classification: [Condition Name]
2. Justification: [Clinical Findings]'''

# Inference function
def analyze_xray(image: Image.Image):
    image = image.convert("RGB")  # Ensure RGB

    # Construct chat prompt
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": instruction},
                {"type": "image", "image": image}
            ]
        }
    ]

    # Apply chat template
    prompt = processor.tokenizer.apply_chat_template(
        messages, add_generation_prompt=True, tokenize=False
    )

    # Preprocess
    inputs = processor(
        text=prompt,
        images=image,
        return_tensors="pt"
    ).to("cuda", torch.float16)

    # Generate output
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=300,
            use_cache=True,
            temperature=1.2,
            top_p=0.9
        )

    # Decode output
    decoded = processor.tokenizer.decode(output_ids[0], skip_special_tokens=False)
    assistant_part = decoded.split("<|start_header_id|>assistant<|end_header_id|>")[1]
    result = assistant_part.split("<|eot_id|>")[0].strip()
    return result

# Example images
example_images = [
    ["/content/Normal.jpg"],
    ["/content/Pneumonia.jpg"],
    ["/content/covid.jpg"]
]

# Create Gradio UI
iface = gr.Interface(
    fn=analyze_xray,
    inputs=gr.Image(type="pil", label="Upload Chest X-ray"),
    outputs=gr.Textbox(label="Model Analysis"),
    title="Chest X-ray Diagnosis (AI Radiologist)",
    description="Upload a chest X-ray to classify it as COVID-19, Pneumonia (non-COVID), or Normal. The model provides justification based on visual findings.",
    examples=example_images
)

# Launch the app
iface.launch(pwa=True)


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ea2c3505bca01b4075.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
